In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver

import re
import time

# 다이닝코드

In [2]:
D_restaurants_df = pd.DataFrame(columns=['name','address','category','main_mn','price','opng_tm','rating','rvw_cnt','tags'])
D_review_df = pd.DataFrame(columns=['res_name','user_name','rating','review'])

In [3]:
with open("../data/restaurants_DiningCode.txt","r") as f:
    text = f.read()
    urls = text.split()

In [4]:
urls

['https://www.diningcode.com/profile.php?rid=PYk33Cv6jpwY',
 'https://www.diningcode.com/profile.php?rid=HnGIlmCt6tYi',
 'https://www.diningcode.com/profile.php?rid=yJWYroWneBZz',
 'https://www.diningcode.com/profile.php?rid=WDLc5zBWbJ0K',
 'https://www.diningcode.com/profile.php?rid=aHS08fgvAQtp',
 'https://www.diningcode.com/profile.php?rid=fAaYVH7bahro',
 'https://www.diningcode.com/profile.php?rid=OdXIdESEcqRN',
 'https://www.diningcode.com/profile.php?rid=VwPxBz81xCXR',
 'https://www.diningcode.com/profile.php?rid=xo80uBfS9kKu',
 'https://www.diningcode.com/profile.php?rid=S3xWF4cDCXZ3',
 'https://www.diningcode.com/profile.php?rid=njPwkrsv7oFr',
 'https://www.diningcode.com/profile.php?rid=tQjzQKd3R2jR',
 'https://www.diningcode.com/profile.php?rid=Dw5rZ87WVOLS',
 'https://www.diningcode.com/profile.php?rid=GxkWR6KdjVZh',
 'https://www.diningcode.com/profile.php?rid=4tnoUOSjdKC9',
 'https://www.diningcode.com/profile.php?rid=OnG6ipd80ZG3',
 'https://www.diningcode.com/profile.php

In [5]:
def crawling_reviews_DiningCode(name, link):
    reviews = pd.DataFrame(columns=['res_name','user_name','rating','review'])
    try:
        driver.get(link)
    except:
        return None
    
    while True:
        try:
            click_more = driver.find_element_by_xpath("""//*[@id="div_more_review"]""")
            click_more.click()
        except:
            break
            
    time.sleep(1)
    
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        for user in soup.find('div',{'id':'div_review'}).find_all('div',{'id':re.compile('div_review_+')}):
            tmp = user.find('span','btxt').get_text()
            user_name = tmp[:tmp.find('(')].strip()
            user_rating = int(re.findall('\d+',user.find('i',{'style':re.compile('width:+')})['style'])[0])//20
            user_review = user.find('p','review_contents btxt').get_text().replace('\n','')
        
            reviews = reviews.append({'res_name':name,
                                    'user_name':user_name,
                                    'rating':user_rating,
                                    'review':user_review}, ignore_index=True)
    except:
        return None
    
    
    return reviews

In [6]:
D_restaurants_df

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags


In [7]:
D_review_df

,res_name,user_name,rating,review


In [8]:
def crawling_res_data_DiningCode(link):
    
    #링크 불러오기
    try:
        html = urlopen(link)
        soup = BeautifulSoup(html, "html.parser")
    except:
        print('링크 에러:', link)
        return None,None,None,None,None,None,None,None,None,None
    
    #상호명
    try:
        name=soup.find_all('p','tit')[3].get_text()
        print(name)
    except:
        name=None
    
    #주소
    try:
        address = soup.find('li','locat').get_text()
    except:
        address = None
    
    #업종(다이닝코드엔 업종이 없음)
    category = None
    
    #주메뉴
    try:
        main_menu = soup.find('div','menu-info short').find_all('p','l-txt')[0].get_text().strip()
    except:
        main_menu = None
    
    #가격(주메뉴)
    try:
        price = int(soup.find('div','menu-info short').find_all('p','r-txt')[0].get_text()[:-1].replace(',',''))
    except:
        price = None
    
    #영업시간
    try:
        time_dict = dict()
        opening_time_list = soup.find('div','busi-hours short').find_all('p')[1:]
        for i in range(len(opening_time_list))[0::2]:
            if opening_time_list[i].get_text()!='더보기':
                time_dict[opening_time_list[i].get_text()] = opening_time_list[i+1].get_text()
    
        opening_time = str(time_dict)
    except:
        opening_time = None
    
    #평점과 리뷰 수
    try:
        ratings = soup.find_all('span','point')
        tmp = ratings[0].get_text().strip()
        rating = float(ratings[1].get_text()[:-1])
        review_count = tmp[:tmp.find('명')]
    except:
        rating = None
        review_count = None                 
    
    #태그
    try:
        tags = ','.join([i.get_text()[:i.get_text().find('(')] for i in soup.find_all('p','icon')])
    except:
        tags = None
    
    res_reviews = crawling_reviews_DiningCode(name, url)
    
    return name, address, category, main_menu, price, opening_time, rating, review_count, tags, res_reviews

In [9]:
count = 1
driver = webdriver.Chrome('../driver/chromedriver')

for url in urls:
    print(count,'\t|',end='')
    
    name, address, category, main_menu, price, opening_time, rating, review_count, tags, review_df = crawling_res_data_DiningCode(url)    
    D_restaurants_df = D_restaurants_df.append({'name':name,
                                            'address':address,
                                            'category':category,
                                            'main_mn':main_menu,
                                            'price':price,
                                            'opng_tm':opening_time,
                                            'rating':rating,
                                            'rvw_cnt':review_count,
                                            'tags':tags}, ignore_index=True)

    D_review_df = D_review_df.append(review_df, ignore_index=True)

    count+=1

1 	|중앙해장
2 	|브루클린더버거조인트
3 	|농민백암순대 본점
4 	|다운타우너 청담점
5 	|쮸즈
6 	|미미면가
7 	|어글리스토브
8 	|쉑쉑버거
9 	|하동관 코엑스점
10 	|반룡산
11 	|새벽집
12 	|알베르
13 	|리틀넥
14 	|목포집
15 	|대우식당
16 	|호랑이식당
17 	|진미평양냉면
18 	|대가방
19 	|브라더후드키친
20 	|뽕나무쟁이 족발
21 	|울프강 스테이크하우스
22 	|바비레드 강남본점
23 	|배드파머스
24 	|C27
25 	|을밀대 강남점
26 	|더블트러블
27 	|마녀주방 강남점
28 	|토끼정 본점
29 	|한추
30 	|프리모 바치오바치
31 	|멜팅샵 신사점
32 	|톡톡
33 	|김수사
34 	|외고집설렁탕
35 	|더키친 살바토레 쿠오모 압구정점
36 	|아우어베이커리
37 	|무월식탁
38 	|정식당
39 	|이치에
40 	|무월
41 	|봉밀가
42 	|네기
43 	|클로리스 티룸 신논현점
44 	|훌리오
45 	|에머이 가로수길점
46 	|테라로사 코엑스점
47 	|무차초
48 	|에머이 강남역점
49 	|베러댄비프
50 	|팬아시아
51 	|카페 노티드
52 	|르타오
53 	|갓덴스시
54 	|블랙스테이크 강남점
55 	|구이혁명가 철든놈 제2공장
56 	|브릭오븐
57 	|딸부자네불백
58 	|겟썸커피
59 	|감성타코
60 	|마초쉐프 강남역점
61 	|돈돈 역삼점
62 	|마루가메 제면 강남점
63 	|한일관 압구정점
64 	|매뉴팩트커피
65 	|아그라
66 	|지구당
67 	|창고43 강남점
68 	|강남불백 1호점
69 	|장서는날
70 	|배꼽집
71 	|장정 정
72 	|코나야
73 	|리틀앤머치
74 	|자연은 맛있다
75 	|평양면옥 논현점
76 	|온더보더 코엑스점
77 	|밍글스
78 	|어거스트힐
79 	|땀땀
80 	|낙원타코
81 	|리북집
82 	|구스아일랜드 브루하우스
83 	|우래옥 대치동점
84 	|에이블
85 	|김돈이
86 	|일일향 1호점
87

1207 	|라멘트럭
1208 	|청어람
1209 	|테일러커피 연남1호점
1210 	|맛이차이나
1211 	|잇텐고
1212 	|하카타분코 본점
1213 	|뭄알로이
1214 	|프리모 바치오바치 홍대본점
1215 	|바다회사랑 연남본점
1216 	|무라 1호점
1217 	|아이엠어버거
1218 	|미담진족
1219 	|소년식당
1220 	|피오니
1221 	|아오이토리
1222 	|오레노라멘
1223 	|감칠
1224 	|이치류 홍대본점
1225 	|옥동식
1226 	|빌리프커피
1227 	|감나무집 기사식당
1228 	|이품분식
1229 	|조박집
1230 	|하하
1231 	|라멘베라보
1232 	|박용석스시 1호점
1233 	|성격양식
1234 	|혼가츠
1235 	|스파카나폴리
1236 	|젤라띠 젤라띠
1237 	|교다이야
1238 	|진진
1239 	|또보겠지떡볶이
1240 	|킨지
1241 	|토끼정 홍대점
1242 	|안
1243 	|감성타코
1244 	|진미식당
1245 	|경주식당
1246 	|앤트러사이트 합정점
1247 	|미나리식당
1248 	|콜린
1249 	|파스치노팬스테이크
1250 	|독립카츠
1251 	|크레이지카츠
1252 	|홍마떡
1253 	|굴다리식당 본관
1254 	|마포원조떡볶이
1255 	|옹달샘 본점
1256 	|이치젠
1257 	|벌스
1258 	|초마 홍대점
1259 	|리틀파파
1260 	|동무밥상
1261 	|달콤한 거짓말
1262 	|윤씨그릴방
1263 	|치즈오타쿠 김철수
1264 	|나베식당
1265 	|프릳츠 도화점
1266 	|탄탄면공방 홍대점
1267 	|홍대돈부리 본점
1268 	|바다파스타
1269 	|홍대개미 상수본점
1270 	|봉쥬르하와이
1271 	|테일러커피
1272 	|빚짜
1273 	|이자와 홍대점
1274 	|듀윗
1275 	|은행골
1276 	|동차밥
1277 	|향미
1278 	|부탄츄
1279 	|문
1280 	|김덕후의곱창조
1281 	|커피가게 동경
1282 	|옥타


In [13]:
D_restaurants_df.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags
0,중앙해장,서울특별시 강남구 대치동 996-16,None,양선지해장국,9000,"{'매일': '24시간 영업 ', '일요일': '자정 12시 - 오후 10시 (월요...",4.3,50,"저녁식사,점심식사,술모임,식사모임,가족외식,회식,데이트,혼밥,아침식사,기념일,차모임..."
1,브루클린더버거조인트,서울특별시 강남구 신사동 540-11,None,더 치즈버거,7800,{'매일': '24시간 영업 '},4.2,46,"저녁식사,점심식사,데이트,혼밥,아침식사,식사모임,간식,브런치,남자끼리,가족외식,여자..."
2,농민백암순대 본점,서울시 강남구 대치동 896-33 (알앤지타운 1F),None,국밥,8000,"{'월-금': '오전 11시 - 오후 9시 (11:10분 시작)', '토요일': '...",4.5,36,"점심식사,저녁식사,혼밥,술모임,데이트,가족외식,혼술,식사모임,아침식사,회식,실버푸드..."
3,다운타우너 청담점,서울특별시 강남구 신사동 666-25,None,아보카도 버거,9300,{'매일': '오전 11시 30분 - 오후 9시 30분 (Last Order 20:...,4.5,39,"점심식사,저녁식사,데이트,혼밥,간식,식사모임,여자끼리,가족외식,아침식사,기념일,남자..."
4,쮸즈,서울특별시 강남구 신사동 540-4,None,소룡포,3500,{'매일': '오전 11시 30분 - 오후 9시 (Break time 15:00~1...,4.1,39,"점심식사,저녁식사,데이트,식사모임,혼밥,기념일,가족외식,남자끼리,여자끼리,친구들끼리..."


In [15]:
D_review_df

,res_name,user_name,rating,review
0,중앙해장,rimrim,4,“깔끔한 해장국으로 속 까지 편안하게”보통 내가 접한 양선지해장국은 너무 매콤하거나...
1,중앙해장,멍청한햄스터,4,"음식맛은 국물은 얼큰 깔끔 속이 풀리는 맛이고 곱창은 곱도 차있고 씹으면 고소, 특..."
2,중앙해장,달리는진저맨,4,"삼성역 맛집 탑급 중 하나인 중앙해장일요일 다섯시쯤 방문, 웨이팅은 없었다.첫 인상..."
3,중앙해장,ye,5,"- 수요미식회 맛집, 친구 추천으로 방문- 웨이팅 순번 적고 가게에서 이름 호명될때..."
4,중앙해장,Kunwoo Kim,5,퇴근길 아내와 저녁식사어제 먹은 술이 확 내려갑니다깔끔한 맛 강추!!우족탕 고기가 ...
5,중앙해장,일상다반사,4,매장 뒷편으로 주차장 있음항상 사람 많고 대기 시간 엄청남음식 맛은 매우 훌륭하나 ...
6,중앙해장,Kaanlim,3,주메뉴인 곱창전골은 모르겠으나내장탕은 가격이 비싼거에 비해서 맛이 상당한 편은 아니...
7,중앙해장,hange,4,곱창전골이 유명하대서 웨이팅감수하고 일찍 방문했어요 내부가 넓어서 생각보다 일찍 차...
8,중앙해장,조이,5,일단 가보고 드셔보세요.곱창을 좋아하거나 곱창전골을 좋아하거나 얼큰한 해장국을 좋아...
9,중앙해장,KW,4,곱창전골로 유명한 곳이라던데 확실히 곱창전골이 맛있었다. 하지만 버튼이 없고 사람이...


In [29]:
D_restaurants_df.to_csv('../data/DiningCode_df',index=False)
D_review_df.to_csv('../data/DiningCode_review_df', index=False)

In [30]:
DiningCode_df = pd.read_csv('../data/DiningCode_df')
DiningCode_review_df = pd.read_csv('../data/DiningCode_review_df')

In [33]:
DiningCode_df.shape

(2360, 9)

In [35]:
DiningCode_review_df.shape

(19684, 4)

In [38]:
DiningCode_df.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags
0,중앙해장,서울특별시 강남구 대치동 996-16,NaN,양선지해장국,9000.0,"{'매일': '24시간 영업 ', '일요일': '자정 12시 - 오후 10시 (월요...",4.3,50.0,"저녁식사,점심식사,술모임,식사모임,가족외식,회식,데이트,혼밥,아침식사,기념일,차모임..."
1,브루클린더버거조인트,서울특별시 강남구 신사동 540-11,NaN,더 치즈버거,7800.0,{'매일': '24시간 영업 '},4.2,46.0,"저녁식사,점심식사,데이트,혼밥,아침식사,식사모임,간식,브런치,남자끼리,가족외식,여자..."
2,농민백암순대 본점,서울시 강남구 대치동 896-33 (알앤지타운 1F),NaN,국밥,8000.0,"{'월-금': '오전 11시 - 오후 9시 (11:10분 시작)', '토요일': '...",4.5,36.0,"점심식사,저녁식사,혼밥,술모임,데이트,가족외식,혼술,식사모임,아침식사,회식,실버푸드..."
3,다운타우너 청담점,서울특별시 강남구 신사동 666-25,NaN,아보카도 버거,9300.0,{'매일': '오전 11시 30분 - 오후 9시 30분 (Last Order 20:...,4.5,39.0,"점심식사,저녁식사,데이트,혼밥,간식,식사모임,여자끼리,가족외식,아침식사,기념일,남자..."
4,쮸즈,서울특별시 강남구 신사동 540-4,NaN,소룡포,3500.0,{'매일': '오전 11시 30분 - 오후 9시 (Break time 15:00~1...,4.1,39.0,"점심식사,저녁식사,데이트,식사모임,혼밥,기념일,가족외식,남자끼리,여자끼리,친구들끼리..."


In [37]:
DiningCode_review_df.head(10)

,res_name,user_name,rating,review
0,중앙해장,rimrim,4,“깔끔한 해장국으로 속 까지 편안하게”보통 내가 접한 양선지해장국은 너무 매콤하거나...
1,중앙해장,멍청한햄스터,4,"음식맛은 국물은 얼큰 깔끔 속이 풀리는 맛이고 곱창은 곱도 차있고 씹으면 고소, 특..."
2,중앙해장,달리는진저맨,4,"삼성역 맛집 탑급 중 하나인 중앙해장일요일 다섯시쯤 방문, 웨이팅은 없었다.첫 인상..."
3,중앙해장,ye,5,"- 수요미식회 맛집, 친구 추천으로 방문- 웨이팅 순번 적고 가게에서 이름 호명될때..."
4,중앙해장,Kunwoo Kim,5,퇴근길 아내와 저녁식사어제 먹은 술이 확 내려갑니다깔끔한 맛 강추!!우족탕 고기가 ...
5,중앙해장,일상다반사,4,매장 뒷편으로 주차장 있음항상 사람 많고 대기 시간 엄청남음식 맛은 매우 훌륭하나 ...
6,중앙해장,Kaanlim,3,주메뉴인 곱창전골은 모르겠으나내장탕은 가격이 비싼거에 비해서 맛이 상당한 편은 아니...
7,중앙해장,hange,4,곱창전골이 유명하대서 웨이팅감수하고 일찍 방문했어요 내부가 넓어서 생각보다 일찍 차...
8,중앙해장,조이,5,일단 가보고 드셔보세요.곱창을 좋아하거나 곱창전골을 좋아하거나 얼큰한 해장국을 좋아...
9,중앙해장,KW,4,곱창전골로 유명한 곳이라던데 확실히 곱창전골이 맛있었다. 하지만 버튼이 없고 사람이...


# 망고플레이트

In [44]:
M_restaurants_df = pd.DataFrame(columns=['name','address','category','main_mn','price','opng_tm','rating','rvw_cnt','tags'])
M_review_df = pd.DataFrame(columns=['res_name','user_name','rating','review'])

In [45]:
with open("../data/restaurants_MangoPlate.txt","r") as f:
    text = f.read()
    Mango_urls = text.split()

In [46]:
Mango_urls

['https://www.mangoplate.com/restaurants/Q-jWS81VNn',
 'https://www.mangoplate.com/restaurants/DfK9o3D1ITgg',
 'https://www.mangoplate.com/restaurants/ZTOnVvaxqk',
 'https://www.mangoplate.com/restaurants/zfARB-g1NKRx',
 'https://www.mangoplate.com/restaurants/cmdrcTiO9Mxk',
 'https://www.mangoplate.com/restaurants/UzWhwg-E_D',
 'https://www.mangoplate.com/restaurants/XcJlmGI2nZlD',
 'https://www.mangoplate.com/restaurants/0FwTUkRDbF',
 'https://www.mangoplate.com/restaurants/T_wWrv3Ba6',
 'https://www.mangoplate.com/restaurants/nj3Hk9TB47dT',
 'https://www.mangoplate.com/restaurants/NMH5sADODQ1h',
 'https://www.mangoplate.com/restaurants/WIuttekMVnQS',
 'https://www.mangoplate.com/restaurants/a9cQoViMuUi1',
 'https://www.mangoplate.com/restaurants/NQVqeYukjT51',
 'https://www.mangoplate.com/restaurants/6wf13T_W5eN_',
 'https://www.mangoplate.com/restaurants/COs1JgXzb5',
 'https://www.mangoplate.com/restaurants/0P2lXS1izr',
 'https://www.mangoplate.com/restaurants/HO_mC5TGgTOc',
 'http

In [42]:
len(Mango_urls)

4358

In [5]:
def crawling_reviews_MangoPlate(name, link):
    reviews = pd.DataFrame(columns=['res_name','user_name','rating','review'])
    try:
        driver.get(link)
    except:
        return None
    
    while True:
        try:
            click_more = driver.find_element_by_xpath("""//*[@id="div_more_review"]""")
            click_more.click()
        except:
            break
            
    time.sleep(1)
    
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        for user in soup.find('div',{'id':'div_review'}).find_all('div',{'id':re.compile('div_review_+')}):
            tmp = user.find('span','btxt').get_text()
            user_name = tmp[:tmp.find('(')].strip()
            user_rating = int(re.findall('\d+',user.find('i',{'style':re.compile('width:+')})['style'])[0])//20
            user_review = user.find('p','review_contents btxt').get_text().replace('\n','')
        
            reviews = reviews.append({'res_name':name,
                                    'user_name':user_name,
                                    'rating':user_rating,
                                    'review':user_review}, ignore_index=True)
    except:
        return None
    
    
    return reviews

In [6]:
def crawling_res_data_MangoPlate(link):
    
    #링크 불러오기
    try:
        html = urlopen(link)
        soup = BeautifulSoup(html, "html.parser")
    except:
        print('링크 에러:', link)
        return None,None,None,None,None,None,None,None,None,None
    
    #상호명
    try:
        name=soup.find_all('p','tit')[3].get_text()
        print(name)
    except:
        name=None
    
    #주소
    try:
        address = soup.find('li','locat').get_text()
    except:
        address = None
    
    #업종(다이닝코드엔 업종이 없음)
    category = None
    
    #주메뉴
    try:
        main_menu = soup.find('div','menu-info short').find_all('p','l-txt')[0].get_text().strip()
    except:
        main_menu = None
    
    #가격(주메뉴)
    try:
        price = int(soup.find('div','menu-info short').find_all('p','r-txt')[0].get_text()[:-1].replace(',',''))
    except:
        price = None
    
    #영업시간
    try:
        time_dict = dict()
        opening_time_list = soup.find('div','busi-hours short').find_all('p')[1:]
        for i in range(len(opening_time_list))[0::2]:
            if opening_time_list[i].get_text()!='더보기':
                time_dict[opening_time_list[i].get_text()] = opening_time_list[i+1].get_text()
    
        opening_time = str(time_dict)
    except:
        opening_time = None
    
    #평점과 리뷰 수
    try:
        ratings = soup.find_all('span','point')
        tmp = ratings[0].get_text().strip()
        rating = float(ratings[1].get_text()[:-1])
        review_count = tmp[:tmp.find('명')]
    except:
        rating = None
        review_count = None                 
    
    #태그
    try:
        tags = ','.join([i.get_text()[:i.get_text().find('(')] for i in soup.find_all('p','icon')])
    except:
        tags = None
    
    res_reviews = crawling_reviews_DiningCode(name, url)
    
    return name, address, category, main_menu, price, opening_time, rating, review_count, tags, res_reviews

In [ ]:
count = 1
driver = webdriver.Chrome('../driver/chromedriver')

for url in urls:
    print(count,'\t|',end='')
    
    name, address, category, main_menu, price, opening_time, rating, review_count, tags, review_df = crawling_res_data_DiningCode(url)    
    D_restaurants_df = D_restaurants_df.append({'name':name,
                                            'address':address,
                                            'category':category,
                                            'main_mn':main_menu,
                                            'price':price,
                                            'opng_tm':opening_time,
                                            'rating':rating,
                                            'rvw_cnt':review_count,
                                            'tags':tags}, ignore_index=True)

    D_review_df = D_review_df.append(review_df, ignore_index=True)

    count+=1

In [60]:
count = 1
driver = webdriver.Chrome('../driver/chromedriver')

In [61]:
driver.get('https://www.mangoplate.com/restaurants/0FwTUkRDbF')

In [62]:
more_count = 0
while (more_count<=20):
    try:
        time.sleep(2)
        driver.find_element_by_xpath("""/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]""").click()
        more_count+=1
        time.sleep(2)
    except:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        if soup.find('div',{'id':'mp20_gallery','class':'on'}) != None:
            driver.find_element_by_xpath("""//*[@id="mp20_gallery"]/div[3]/button/i""")
            continue
        break
        
tabs = driver.window_handles
if len(tabs)>=2:
    for i in range(len(tabs))[:0:-1]:
        driver.switch_to.window(tabs[i])
        driver.close()
        
driver.switch_to.window(tabs[0])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=80.0.3987.163)


In [49]:
driver.find_element_by_class_name('on')

<selenium.webdriver.remote.webelement.WebElement (session="c6ce57c4d571360d724259043588da1b", element="d7e32149-21be-4db4-aaa7-bff114e2c1fa")>

In [24]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [26]:
type(soup.find('div',{'id':'mp20_gallery','class':'on'}))

NoneType

In [14]:
soup.find_all('i','close_icon')

[<i class="close_icon"></i>]

In [52]:
soup.find('h1','restaurant_name').get_text()

'서촌계단집'

In [53]:
soup.find('div','Restaurant__InfoItemLabel--RoadAddressText').get_text()

'서울특별시 종로구 자하문로1길 15'

In [54]:
soup.select('tbody > tr')[2].span.get_text()

'해산물 요리'

In [55]:
soup.find_all('span','Restaurant_Menu')[0].get_text()

'벌교왕꼬막'

In [56]:
int(soup.find_all('span','Restaurant_MenuPrice')[0].get_text()[:-1].replace(',',''))

15000

In [57]:
soup.select('tbody > tr')[5].td.get_text()

'16:00 - 04:00'

In [58]:
float(soup.find('strong','rate-point').span.get_text())

4.3

In [59]:
int(soup.find('span','cnt review').get_text())

137

In [60]:
tags = ''
for tag in soup.find('section','module related-tags only-desktop').find_all('a'):
    tags += tag.get_text().strip()
    tags += ','
tags

'해물,'

In [78]:
users = soup.find_all('a','RestaurantReviewItem__Link')
rating_score = {'맛있다':5,'괜찮다':3,'별로':1}
for user in users:
    user_name = user.span.get_text()
    user_review = user.p.get_text().strip()
    user_rating = user.find('span','RestaurantReviewItem__RatingText').get_text()
    user_rating = rating_score[user_rating]

In [79]:
user_name

'MJ'

In [80]:
user_review

'주말에 tv에 나온 굴을 보고 꽂힌 친구덕에 가봤습니다 ㅋㅋㅋ 전부터 궁금했는데 드디어! 맛집 답게 웨이팅이 있었어요 평일 일곱시 정도였는데 이십분 정도 기다린거 같네요 친구말론 옛날보다 많이 확장해서 넓어졌다고 하는데 그래도 정말 붐볐어요...! 친구들에게 대접할 일이 있어(?) 맘껏 시켜! 하구서 정말 맘껏 시킨 굴과 소라와 문어! 정~말 하나같이 소주를 부르는 안주였는데 요즘 몸이 안좋아 자제중이라 맥주만 가볍게 한게 못내 아쉽습니다 ㅠㅠ 다 맛있었는데 개인적으론 소라가 가장 쫄깃하고 맛있었어요 어흑... 정말 셋이서 저걸 다 ㅋㅋㅋ 먹었는데 해산물로 배터지게 먹은건 오랜만이네요 갑오징어랑 꼬막도 먹어보고 싶었다는..! 으헝 ㅋㅋㅋ 가격은 싸진 않지만 딱 적정선이었던거 같아요 맛있게 먹어서 좋았어요 친구들이 너무 좋아해서 뿌듯.... 웨이팅 감안하면 강추하기엔 망설여지기도 하지만 서촌에서 해산물과 함께 한잔하고 싶을 때 가면 좋을 집!'

In [81]:
user_rating

5